# Import core libraries for data manipulation and visualization


In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)


# Load sales pipeline dataset for exploratory analysis


In [24]:
df = pd.read_csv('skygeni_sales_data.csv')

df.head()


,deal_id,created_date,closed_date,sales_rep_id,industry,region,product_type,lead_source,deal_stage,deal_amount,sales_cycle_days,outcome
0,D00001,2023-11-24,2023-12-15,rep_22,SaaS,North America,Enterprise,Referral,Qualified,4253,21,Won
1,D00002,2023-01-17,2023-01-27,rep_7,SaaS,India,Core,Referral,Closed,3905,10,Won
2,D00003,2023-10-29,2023-12-10,rep_5,HealthTech,APAC,Core,Inbound,Proposal,10615,42,Lost
3,D00004,2023-07-14,2023-08-02,rep_18,FinTech,India,Core,Partner,Negotiation,4817,19,Won
4,D00005,2024-02-29,2024-05-26,rep_2,HealthTech,APAC,Core,Outbound,Qualified,45203,87,Lost


# Inspect dataset structure, data types, and completeness


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   deal_id           5000 non-null   object
 1   created_date      5000 non-null   object
 2   closed_date       5000 non-null   object
 3   sales_rep_id      5000 non-null   object
 4   industry          5000 non-null   object
 5   region            5000 non-null   object
 6   product_type      5000 non-null   object
 7   lead_source       5000 non-null   object
 8   deal_stage        5000 non-null   object
 9   deal_amount       5000 non-null   int64 
 10  sales_cycle_days  5000 non-null   int64 
 11  outcome           5000 non-null   object
dtypes: int64(2), object(10)
memory usage: 468.9+ KB


# Perform initial data quality checks including shape, missing values, and statistical summary


In [26]:
df.shape
df.info()
df.isnull().sum()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   deal_id           5000 non-null   object
 1   created_date      5000 non-null   object
 2   closed_date       5000 non-null   object
 3   sales_rep_id      5000 non-null   object
 4   industry          5000 non-null   object
 5   region            5000 non-null   object
 6   product_type      5000 non-null   object
 7   lead_source       5000 non-null   object
 8   deal_stage        5000 non-null   object
 9   deal_amount       5000 non-null   int64 
 10  sales_cycle_days  5000 non-null   int64 
 11  outcome           5000 non-null   object
dtypes: int64(2), object(10)
memory usage: 468.9+ KB


,deal_amount,sales_cycle_days
count,5000.000000,5000.000000
mean,26286.492800,63.751800
std,27689.230136,32.731405
min,2002.000000,7.000000
25%,6611.000000,35.750000
50%,14171.500000,64.000000
75%,39062.250000,92.000000
max,100000.000000,120.000000


# Convert date columns to datetime format for time-based feature engineering


In [27]:
df['created_date'] = pd.to_datetime(df['created_date'])
df['closed_date'] = pd.to_datetime(df['closed_date'])

# Derive sales cycle duration to capture deal velocity


In [28]:
df['sales_cycle_days'] = (df['closed_date'] - df['created_date']).dt.days

# Analyze win-loss distribution to understand baseline conversion rate


In [29]:
df['outcome'].value_counts(normalize=True) * 100

,proportion
outcome,
Lost,54.74
Won,45.26


# Evaluate win rates across regions to detect geographic performance patterns


In [30]:
pd.crosstab(df['region'], df['outcome'], normalize='index') * 100

outcome,Lost,Won
region,,
APAC,55.072464,44.927536
Europe,54.420114,45.579886
India,54.276827,45.723173
North America,55.205811,44.794189


# Compare average deal value between won and lost opportunities

In [31]:
df.groupby('outcome')['deal_amount'].mean()

,deal_amount
outcome,
Lost,25883.516989
Won,26773.874503


# Assess whether sales cycle length influences deal success


In [32]:
df.groupby('outcome')['sales_cycle_days'].mean()

,sales_cycle_days
outcome,
Lost,64.230179
Won,63.173221


# Examine conversion rates by industry to identify high-performing segments


In [33]:
pd.crosstab(df['industry'], df['outcome'], normalize='index') * 100

outcome,Lost,Won
industry,,
Ecommerce,55.094340,44.905660
EdTech,55.846774,44.153226
FinTech,52.294557,47.705443
HealthTech,55.445545,44.554455
SaaS,54.845155,45.154845


# Analyze lead sources to understand channels driving stronger conversions


In [34]:
pd.crosstab(df['lead_source'], df['outcome'], normalize='index') * 100

outcome,Lost,Won
lead_source,,
Inbound,53.961965,46.038035
Outbound,54.494382,45.505618
Partner,56.048387,43.951613
Referral,54.472843,45.527157


# Review distribution of deals across pipeline stages


In [35]:
pd.crosstab(df['product_type'], df['outcome'], normalize='index') * 100

outcome,Lost,Won
product_type,,
Core,54.486423,45.513577
Enterprise,55.030675,44.969325
Pro,54.713604,45.286396


# Measure win rates by deal stage to evaluate pipeline progression quality


The dataset shows a moderate conversion rate (~45%), indicating an opportunity to improve sales effectiveness.

In [36]:
df['deal_stage'].value_counts()

,count
deal_stage,
Demo,1043
Proposal,1009
Closed,997
Negotiation,995
Qualified,956


# Estimate pipeline quality by analyzing late-stage conversion rates


In [37]:
pd.crosstab(df['deal_stage'], df['outcome'], normalize='index') * 100

outcome,Lost,Won
deal_stage,,
Closed,53.259779,46.740221
Demo,54.170662,45.829338
Negotiation,53.366834,46.633166
Proposal,55.302279,44.697721
Qualified,57.740586,42.259414


# Create revenue-per-day metric to evaluate sales efficiency


In [38]:
late_stages = ['Negotiation', 'Closed']

late_stage_deals = df[df['deal_stage'].isin(late_stages)]

pipeline_quality = (
    late_stage_deals['outcome'].value_counts(normalize=True) * 100
)

pipeline_quality

,proportion
outcome,
Lost,53.313253
Won,46.686747


In [44]:
df.columns

Index(['deal_id', 'created_date', 'closed_date', 'sales_rep_id', 'industry',
       'region', 'product_type', 'lead_source', 'deal_stage', 'deal_amount',
       'sales_cycle_days', 'outcome'],
      dtype='object')

In [49]:
df['revenue_per_day'] = df['deal_amount'] / df['sales_cycle_days']

# Segment deals into size tiers to study conversion behavior by deal magnitude


In [50]:
df['revenue_per_day'] = df['deal_amount'] / df['sales_cycle_days']
df.groupby('outcome')['revenue_per_day'].mean()

,revenue_per_day
outcome,
Lost,631.207376
Won,690.006599


# Encode outcome as a binary target variable for modeling


In [51]:
df.groupby('outcome')['revenue_per_day'].mean().round(2)

,revenue_per_day
outcome,
Lost,631.21
Won,690.01


# Separate predictive features from target while removing non-informative identifiers


In [55]:
df['outcome_flag'] = df['outcome'].map({'Won': 1, 'Lost': 0})
features = df.drop(columns=[
    'deal_id',
    'created_date',
    'closed_date',
    'outcome',
    'outcome_flag'
])
target = df['outcome_flag']


# Apply one-hot encoding to categorical variables for model compatibility


In [56]:
df.head()

,deal_id,created_date,closed_date,sales_rep_id,industry,region,product_type,lead_source,deal_stage,deal_amount,sales_cycle_days,outcome,revenue_per_day,outcome_flag
0,D00001,2023-11-24,2023-12-15,rep_22,SaaS,North America,Enterprise,Referral,Qualified,4253,21,Won,202.523810,1
1,D00002,2023-01-17,2023-01-27,rep_7,SaaS,India,Core,Referral,Closed,3905,10,Won,390.500000,1
2,D00003,2023-10-29,2023-12-10,rep_5,HealthTech,APAC,Core,Inbound,Proposal,10615,42,Lost,252.738095,0
3,D00004,2023-07-14,2023-08-02,rep_18,FinTech,India,Core,Partner,Negotiation,4817,19,Won,253.526316,1
4,D00005,2024-02-29,2024-05-26,rep_2,HealthTech,APAC,Core,Outbound,Qualified,45203,87,Lost,519.574713,0


# Encode deal outcome as a binary target variable for supervised modeling


In [57]:
import pandas as pd

# Target
df['outcome_flag'] = df['outcome'].map({'Lost':0, 'Won':1})

# Features + Target
X = df.drop(columns=[
    'deal_id',
    'created_date',
    'closed_date',
    'outcome',
    'outcome_flag'
])

y = df['outcome_flag']

# Encoding
X = pd.get_dummies(X, drop_first=True)

print("Shape of X:", X.shape)


Shape of X: (5000, 43)


# Split the dataset into training and testing sets to evaluate model performance on unseen data


In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# Initialize and train a Random Forest classifier to predict deal outcomes based on sales attributes


In [59]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=200,
    max_depth=6,
    random_state=42
)

model.fit(X_train, y_train)

RandomForestClassifier(max_depth=6, n_estimators=200, random_state=42)

# Identify the most influential features driving deal success or failure using feature importance scores


In [60]:
importance = pd.Series(
    model.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

importance.head(10)


,0
revenue_per_day,0.180948
deal_amount,0.160408
sales_cycle_days,0.148614
product_type_Pro,0.026070
deal_stage_Qualified,0.023709
deal_stage_Demo,0.023013
lead_source_Referral,0.021468
industry_FinTech,0.021401
region_India,0.020973
lead_source_Partner,0.020441


# Evaluate model accuracy on the test dataset to measure predictive performance


In [61]:
model.score(X_test, y_test)

0.555

# Train a Random Forest classifier to capture nonlinear relationships and generate directional predictions


In [62]:
from sklearn.metrics import classification_report

pred = model.predict(X_test)

print(classification_report(y_test, pred))


              precision    recall  f1-score   support

           0       0.56      0.96      0.71       560
           1       0.44      0.04      0.07       440

    accuracy                           0.56      1000
   macro avg       0.50      0.50      0.39      1000
weighted avg       0.51      0.56      0.43      1000



# Extract feature importance to identify the primary drivers influencing deal outcomes


In [63]:
risk_probs = model.predict_proba(X_test)[:,0]   # probability of LOSS

risk_df = pd.DataFrame({
    'Risk_of_Loss': risk_probs
})

risk_df.head()

,Risk_of_Loss
0,0.525132
1,0.557290
2,0.515158
3,0.561569
4,0.540172


# Evaluate baseline accuracy to measure overall predictive signal


In [64]:
risk_df['Risk_Category'] = pd.cut(
    risk_df['Risk_of_Loss'],
    bins=[0, 0.4, 0.7, 1],
    labels=['Low Risk', 'Medium Risk', 'High Risk']
)

risk_df.head(10)


,Risk_of_Loss,Risk_Category
0,0.525132,Medium Risk
1,0.557290,Medium Risk
2,0.515158,Medium Risk
3,0.561569,Medium Risk
4,0.540172,Medium Risk
5,0.572648,Medium Risk
6,0.520225,Medium Risk
7,0.552660,Medium Risk
8,0.573458,Medium Risk
9,0.549973,Medium Risk


# Generate detailed classification metrics to better understand prediction behavior across classes

In [65]:
risk_df['Risk_Category'].value_counts()


,count
Risk_Category,
Medium Risk,1000
Low Risk,0
High Risk,0


# Estimate probability of deal loss to support risk-based prioritization

In [66]:
final_risk_view = X_test.copy()

final_risk_view['Actual_Outcome'] = y_test.values
final_risk_view['Risk_of_Loss'] = risk_probs
final_risk_view['Risk_Category'] = risk_df['Risk_Category'].values

final_risk_view.head()


,deal_amount,sales_cycle_days,revenue_per_day,sales_rep_id_rep_10,sales_rep_id_rep_11,sales_rep_id_rep_12,sales_rep_id_rep_13,sales_rep_id_rep_14,sales_rep_id_rep_15,sales_rep_id_rep_16,sales_rep_id_rep_17,sales_rep_id_rep_18,sales_rep_id_rep_19,sales_rep_id_rep_2,sales_rep_id_rep_20,sales_rep_id_rep_21,sales_rep_id_rep_22,sales_rep_id_rep_23,sales_rep_id_rep_24,sales_rep_id_rep_25,sales_rep_id_rep_3,sales_rep_id_rep_4,sales_rep_id_rep_5,sales_rep_id_rep_6,sales_rep_id_rep_7,sales_rep_id_rep_8,sales_rep_id_rep_9,industry_EdTech,industry_FinTech,industry_HealthTech,industry_SaaS,region_Europe,region_India,region_North America,product_type_Enterprise,product_type_Pro,lead_source_Outbound,lead_source_Partner,lead_source_Referral,deal_stage_Demo,deal_stage_Negotiation,deal_stage_Proposal,deal_stage_Qualified,Actual_Outcome,Risk_of_Loss,Risk_Category
1501,98900,111,890.990991,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,True,False,True,False,False,False,0,0.525132,Medium Risk
2586,17268,99,174.424242,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,True,True,False,False,False,False,False,False,1,0.557290,Medium Risk
2653,95985,65,1476.692308,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,0,0.515158,Medium Risk
1055,71407,44,1622.886364,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,0,0.561569,Medium Risk
705,12255,21,583.571429,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True,False,False,False,False,False,False,True,0,0.540172,Medium Risk


# Segment deals into risk tiers to improve leadership visibility and intervention planning

In [67]:
risk_df['Risk_Category'] = pd.qcut(
    risk_df['Risk_of_Loss'],
    q=3,
    labels=['Low Risk', 'Medium Risk', 'High Risk']
)

risk_df['Risk_Category'].value_counts()


,count
Risk_Category,
Low Risk,334
Medium Risk,333
High Risk,333


# Analyze distribution of predicted risk categories to assess probability separation

In [68]:
final_risk_view.sort_values(
    by='Risk_of_Loss',
    ascending=False
).head(10)


,deal_amount,sales_cycle_days,revenue_per_day,sales_rep_id_rep_10,sales_rep_id_rep_11,sales_rep_id_rep_12,sales_rep_id_rep_13,sales_rep_id_rep_14,sales_rep_id_rep_15,sales_rep_id_rep_16,sales_rep_id_rep_17,sales_rep_id_rep_18,sales_rep_id_rep_19,sales_rep_id_rep_2,sales_rep_id_rep_20,sales_rep_id_rep_21,sales_rep_id_rep_22,sales_rep_id_rep_23,sales_rep_id_rep_24,sales_rep_id_rep_25,sales_rep_id_rep_3,sales_rep_id_rep_4,sales_rep_id_rep_5,sales_rep_id_rep_6,sales_rep_id_rep_7,sales_rep_id_rep_8,sales_rep_id_rep_9,industry_EdTech,industry_FinTech,industry_HealthTech,industry_SaaS,region_Europe,region_India,region_North America,product_type_Enterprise,product_type_Pro,lead_source_Outbound,lead_source_Partner,lead_source_Referral,deal_stage_Demo,deal_stage_Negotiation,deal_stage_Proposal,deal_stage_Qualified,Actual_Outcome,Risk_of_Loss,Risk_Category
228,18103,99,182.858586,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,True,False,True,False,False,False,False,True,0,0.610268,Medium Risk
4253,9138,114,80.157895,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,True,False,0,0.608482,Medium Risk
4615,17920,103,173.980583,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,True,1,0.605505,Medium Risk
691,18188,67,271.462687,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,True,0,0.599729,Medium Risk
577,51079,106,481.877358,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,True,False,False,1,0.598115,Medium Risk
1321,6653,63,105.603175,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,True,False,False,True,False,False,False,True,False,1,0.597254,Medium Risk
3288,7592,120,63.266667,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,1,0.595268,Medium Risk
4741,6869,44,156.113636,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,True,False,False,False,True,1,0.595097,Medium Risk
544,8447,44,191.977273,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,True,False,True,False,False,False,False,True,0,0.594380,Medium Risk
1295,50821,97,523.927835,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,False,0,0.594257,Medium Risk


# Combine predictions with deal data to create an analyst-ready risk view for decision support

In [69]:
importance.head(5)


,0
revenue_per_day,0.180948
deal_amount,0.160408
sales_cycle_days,0.148614
product_type_Pro,0.026070
deal_stage_Qualified,0.023709
